<a href="https://colab.research.google.com/github/Paulina9555/kursGIT/blob/master/Kopia_notatnika_Train_sentence_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq sentence_transformers==2.2.2

In [ ]:
# from termcolor import colored
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from sentence_transformers.util import cos_sim
from tqdm import tqdm

from transformers import logging
logging.set_verbosity_error()

In [ ]:
df = pd.read_csv("konstytucja.csv")
train_sentences  = df['content'].str.lower().to_list()

In [ ]:
df

,Unnamed: 0,content,idx
0,0,Akt prawny©Kancelaria Sejmu s. 1/56 2015-01-0...,0
1,1,1. Rzeczpospolita Polska jest dobrem wspólnym...,1
2,2,2. Rzeczpospolita Polska jest demokratycznym ...,2
3,3,3. Rzeczpospolita Polska jest państwem jednol...,3
4,4,4. 1. Władza zwierzchnia w Rzeczypospolitej P...,4
...,...,...,...
239,239,239. 1. W okresie 2 lat od dnia wejścia w życ...,239
240,240,240. W okresie roku od dnia wejścia w życie K...,240
241,241,241. 1. Umowy międzynarodowe ratyfikowane dot...,241
242,242,242. Tracą moc: 1) ustawa konstytucyjna z dni...,242


In [ ]:
from IPython.display import HTML as html_print

def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

In [ ]:
class TrainSbert:
    def __init__(self, model_name, train_sentences, output_path, batch_size = 4, epochs = 10 ):
        self.model_name = model_name
        self.batch_size = batch_size
        self.epochs = epochs
        self.train_sentences = train_sentences
        self.output_path = output_path
        self.model = self.load_model()
        self.train_dataloader = self.load_data()
        self.train_loss = self.load_loss()

    def load_model(self):
        word_embedding_model = models.Transformer(self.model_name)
        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
        model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
        return model

    def load_data(self):
        train_dataset = datasets.DenoisingAutoEncoderDataset(self.train_sentences)
        train_dataloader = DataLoader(train_dataset, batch_size=self.batch_size , shuffle=True)
        return train_dataloader

    def load_loss(self):
        train_loss = losses.DenoisingAutoEncoderLoss(self.model, decoder_name_or_path=self.model_name, tie_encoder_decoder=True)
        return train_loss

    def train(self):
        self.model.fit(
                train_objectives=[(self.train_dataloader, self.train_loss)],
                epochs=self.epochs,
                weight_decay=0,
                scheduler='constantlr',
                optimizer_params={'lr': 3e-5},
                show_progress_bar=True,
                save_best_model = True,
                output_path = self.output_path
                )


In [ ]:
def load_model(model_path):
    word_embedding_model = models.Transformer(model_path)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model

def generate_emb(model, data):
    emb = []
    for doc in tqdm(data):
        emb.append(model.encode(doc))
    return emb

def get_answer(query, models_and_emb, baza, top = 3):
    for name, model in models_and_emb.items():

        display(html_print(cstr(name, color='green')))
        query_emb = model[0].encode([query])

        cosine = []
        for idx, emm_doc in enumerate(model[1]):
            cosine.append(cos_sim(query_emb, emm_doc).cpu().numpy()[0][0])

        idx = sorted(range(len(cosine)), key=lambda i: cosine[i])[-top:]

        display(html_print(cstr(query, color='red')))
        print("--"*40)
        odp = []
        for doc_idx in idx[::-1]:
            print(f"Doc id: { doc_idx}")
            print(cosine[doc_idx].round(3))
            print(baza[doc_idx])
            odp.append(baza[doc_idx])
            print("--"*10)

## Sdadas

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
args = {
    "model_name":'sdadas/mmlw-retrieval-e5-base',
    "train_sentences": train_sentences,
    "output_path":"models/FT_30epoch_split_data/sdadas/mmlw-retrieval-e5-base",
    "batch_size" : 4,
    "epochs" : 20}

sdadas = TrainSbert(**args)
sdadas.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/556M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/61 [00:00<?, ?it/s]

Iteration:   0%|          | 0/61 [00:00<?, ?it/s]

Iteration:   0%|          | 0/61 [00:00<?, ?it/s]

Iteration:   0%|          | 0/61 [00:00<?, ?it/s]

Iteration:   0%|          | 0/61 [00:00<?, ?it/s]

In [ ]:
### Z HF
sdadas_FT = load_model("models/FT_30epoch_split_data/sdadas/mmlw-retrieval-e5-base")
emb_FT = generate_emb(sdadas_FT, train_sentences)

100%|██████████| 244/244 [00:07<00:00, 34.80it/s]


In [ ]:
sdadas = load_model('models/FT_30epoch_split_data/sdadas/mmlw-retrieval-e5-base')
emb = generate_emb(sdadas, train_sentences)

100%|██████████| 244/244 [00:04<00:00, 53.08it/s]


In [ ]:
models_and_emb = {"sdadas": [sdadas, emb],
                  "sdadas FT": [sdadas_FT, emb_FT]}

In [ ]:
questions = [
            "Z ilu posłów składa sie sejm?",
            "Z ilu posłów składa się senat?",
            "Kto może zostać prezydentem?",
            "Jak długo trwa kadencja sejmu"
            ]


for question in questions:
    get_answer(question, models_and_emb, baza = train_sentences)

--------------------------------------------------------------------------------
Doc id: 96
0.691
 96. 1. sejm składa się z 460 posłów. 2. wybory do sejmu są powszechne, równe, bezpośrednie i proporcjonalne oraz odbywają się w głosowaniu tajnym. 
--------------------
Doc id: 102
0.674
 102. nie można być równocześnie posłem i senatorem. 
--------------------
Doc id: 113
0.668
 113. posiedzenia sejmu są jawne. jeżeli wymaga tego dobro państwa, sejm może bezwzględną większością głosów w obecności co najmniej połowy ustawowej liczby posłów uchwalić tajność obrad. 
--------------------


--------------------------------------------------------------------------------
Doc id: 96
0.691
 96. 1. sejm składa się z 460 posłów. 2. wybory do sejmu są powszechne, równe, bezpośrednie i proporcjonalne oraz odbywają się w głosowaniu tajnym. 
--------------------
Doc id: 102
0.674
 102. nie można być równocześnie posłem i senatorem. 
--------------------
Doc id: 113
0.668
 113. posiedzenia sejmu są jawne. jeżeli wymaga tego dobro państwa, sejm może bezwzględną większością głosów w obecności co najmniej połowy ustawowej liczby posłów uchwalić tajność obrad. 
--------------------


--------------------------------------------------------------------------------
Doc id: 97
0.738
 97. 1. senat składa się ze 100 senatorów. 2. wybory do senatu są powszechne, bezpośrednie i odbywają się w głosowaniu tajnym. 
--------------------
Doc id: 96
0.692
 96. 1. sejm składa się z 460 posłów. 2. wybory do sejmu są powszechne, równe, bezpośrednie i proporcjonalne oraz odbywają się w głosowaniu tajnym. 
--------------------
Doc id: 102
0.681
 102. nie można być równocześnie posłem i senatorem. 
--------------------


--------------------------------------------------------------------------------
Doc id: 97
0.738
 97. 1. senat składa się ze 100 senatorów. 2. wybory do senatu są powszechne, bezpośrednie i odbywają się w głosowaniu tajnym. 
--------------------
Doc id: 96
0.692
 96. 1. sejm składa się z 460 posłów. 2. wybory do sejmu są powszechne, równe, bezpośrednie i proporcjonalne oraz odbywają się w głosowaniu tajnym. 
--------------------
Doc id: 102
0.681
 102. nie można być równocześnie posłem i senatorem. 
--------------------


--------------------------------------------------------------------------------
Doc id: 102
0.633
 102. nie można być równocześnie posłem i senatorem. 
--------------------
Doc id: 127
0.601
 127. 1. prezydent rzeczypospolitej jest wybierany przez naród w wyborach powszechnych, równych, bezpośrednich i w głosowaniu tajnym. 2. prezydent rzeczypospolitej jest wybierany na pięcioletnią kadencję i może być ponownie wybrany tylko raz. ©kancelaria sejmu  s. 27/56 2015-01-07 3. na prezydenta rzeczypospolitej może być wybrany obywatel polski, który najpóźniej w dniu wyborów kończy 35 lat i korzysta z pełni praw wyborczych do sejmu. kandydata zgłasza co najmniej 100 000 obywateli mających prawo wybierania do sejmu. 4. na prezydenta rzeczypospolitej wybrany zostaje kandydat, który otrzymał więcej niż połowę ważnie oddanych głosów. jeżeli żaden z kandydatów nie uzyska wymaganej większości, czternastego dnia po pierwszym głosowaniu przeprowadza się ponowne głosowanie. 5. w ponownym głosowaniu wyb

--------------------------------------------------------------------------------
Doc id: 102
0.633
 102. nie można być równocześnie posłem i senatorem. 
--------------------
Doc id: 127
0.601
 127. 1. prezydent rzeczypospolitej jest wybierany przez naród w wyborach powszechnych, równych, bezpośrednich i w głosowaniu tajnym. 2. prezydent rzeczypospolitej jest wybierany na pięcioletnią kadencję i może być ponownie wybrany tylko raz. ©kancelaria sejmu  s. 27/56 2015-01-07 3. na prezydenta rzeczypospolitej może być wybrany obywatel polski, który najpóźniej w dniu wyborów kończy 35 lat i korzysta z pełni praw wyborczych do sejmu. kandydata zgłasza co najmniej 100 000 obywateli mających prawo wybierania do sejmu. 4. na prezydenta rzeczypospolitej wybrany zostaje kandydat, który otrzymał więcej niż połowę ważnie oddanych głosów. jeżeli żaden z kandydatów nie uzyska wymaganej większości, czternastego dnia po pierwszym głosowaniu przeprowadza się ponowne głosowanie. 5. w ponownym głosowaniu wyb

--------------------------------------------------------------------------------
Doc id: 243
0.697
 243. konstytucja rzeczypospolitej polskiej wchodzi w życie po upływie 3 miesięcy od dnia jej ogłoszenia.  
--------------------
Doc id: 98
0.67
 98. 1. sejm i senat są wybierane na czteroletnie kadencje. kadencje sejmu i senatu rozpoczynają się z dniem zebrania się sejmu na pierwsze posiedzenie i trwają do dnia poprzedzającego dzień zebrania się sejmu następnej kadencji. ©kancelaria sejmu  s. 20/56 2015-01-07 2. wybory do sejmu i senatu zarządza prezydent rzeczypospolitej nie później niż na 90 dni przed upływem 4 lat od rozpoczęcia kadencji sejmu i senatu, wyznaczając wybory na dzień wolny od pracy, przypadający w ciągu 30 dni przed upływem 4 lat od rozpoczęcia kadencji sejmu i senatu. 3. sejm może skrócić swoją kadencję uchwałą podjętą większością co najmniej 2/3 głosów ustawowej liczby posłów. skrócenie kadencji sejmu oznacza jednoczesne skrócenie kadencji senatu. przepis ust. 5 stosuj

--------------------------------------------------------------------------------
Doc id: 243
0.697
 243. konstytucja rzeczypospolitej polskiej wchodzi w życie po upływie 3 miesięcy od dnia jej ogłoszenia.  
--------------------
Doc id: 98
0.67
 98. 1. sejm i senat są wybierane na czteroletnie kadencje. kadencje sejmu i senatu rozpoczynają się z dniem zebrania się sejmu na pierwsze posiedzenie i trwają do dnia poprzedzającego dzień zebrania się sejmu następnej kadencji. ©kancelaria sejmu  s. 20/56 2015-01-07 2. wybory do sejmu i senatu zarządza prezydent rzeczypospolitej nie później niż na 90 dni przed upływem 4 lat od rozpoczęcia kadencji sejmu i senatu, wyznaczając wybory na dzień wolny od pracy, przypadający w ciągu 30 dni przed upływem 4 lat od rozpoczęcia kadencji sejmu i senatu. 3. sejm może skrócić swoją kadencję uchwałą podjętą większością co najmniej 2/3 głosów ustawowej liczby posłów. skrócenie kadencji sejmu oznacza jednoczesne skrócenie kadencji senatu. przepis ust. 5 stosuj